In [1]:
import dotenv
import os
import replicate

dotenv.load_dotenv()

True

In [2]:
replicate_api_token = os.getenv('REPLICATE_API_TOKEN')

In [38]:
style = 'grayscale, detailed drawing, japanese manga\n'
person_description = 'The person has a defined jawline with a slightly angular facial structure, contributing to a strong and masculine appearance. Their skin tone appears smooth and has a warm undertone, suggesting a healthy complexion. The hair is dark and styled in a short, spiky manner, with the sides closely cropped, adding to the overall structured look.\n'
scene = 'He is wearing a smart casual outfit with a vibrant red hoodie and jeans. The setting is a crowded hackathon event in a large hall filled with rows of computers and banners of tech sponsors. The atmosphere is bustling and brightly lit with fluorescent lights overhead.'

In [40]:
prompt = style + person_description + scene

In [42]:
output = replicate.run(
    "black-forest-labs/flux-dev",
    input={
        "prompt": prompt,
        "go_fast": False,
        "guidance": 6,
        "megapixels": "1",
        "num_outputs": 1,
        "aspect_ratio": "1:1",
        "output_quality": 80,
        "num_inference_steps": 28,
    },
)

In [43]:
import base64

base64_string = str(output[0]).replace("data:application/octet-stream;base64,", "")
image_data = base64.b64decode(base64_string)
with open("image.png", "wb") as file:
    file.write(image_data)

In [55]:
from PIL import Image, ImageDraw, ImageFont

image = Image.open('image.png')
draw = ImageDraw.Draw(image)

caption = 'this is a caption'
font_size = 50
try:
    font = ImageFont.truetype("arial.ttf", font_size)
except IOError:
    font = ImageFont.load_default(size=font_size)

bbox = draw.textbbox((0, 0), caption, font=font)
text_width = bbox[2] - bbox[0]
text_height = bbox[3] - bbox[1]
image_width, image_height = image.size
x_position = (image_width - text_width) / 2
y_position = image_height - text_height - 30  # 10 pixels from the bottom
draw.text((x_position, y_position), caption, font=font, fill="black", stroke_fill='white', stroke_width=3)

image.save('captioned_image.png')